In [36]:
import pandas as pd
import numpy as np
import json
import os
from Agents import *

In [37]:
path = "C:/Users/trist/OneDrive/Documents/draft_data_public.VOW.PremierDraft.csv"
path2 = "C:/Users/trist/Downloads/VOW.json"

## Beautiful Soup + Selenium to get new weights from 17lands

In [38]:
from selenium import webdriver 
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

chrome_options = Options()

chrome_options.add_argument("--headless")

archetypes = ['WU','WB','WR','WG','UB','UR','UG','BR','BG','RG']

expansion_string = 'VOW'


#store the url of the first page
url = 'https://www.17lands.com/card_ratings'
#define the executable path on the webdriver using the file's destination 
driver = webdriver.Chrome(executable_path='C:/Users/trist/Downloads/chromedriver.exe',options=chrome_options)
try:
    #get the url in the driver, use either URL or URL_two depending on which page you're trying to scrape
    driver.get(url)

    #Now let's make sure that we are choosing the right set
    select = Select(driver.find_element_by_id('expansion'))

    # select by visible text
    select.select_by_visible_text(expansion_string)

    #Wait for our table body to load
    table = WebDriverWait(driver, 20).until(EC.visibility_of_element_located(
        (By.XPATH, '//tbody')))

    #Grab the page source
    html=driver.page_source

    #5 second cooldown 
    time.sleep(5)

    #Create list of html tables that we will later iterate through
    html_list = []
    for arch in archetypes:
        #Now let's make sure that we are choosing the right set
        select = Select(driver.find_element_by_id('deck_color'))

        # select by visible text
        select.select_by_visible_text(arch)

        #Wait for our table body to load
        table = WebDriverWait(driver, 20).until(EC.visibility_of_element_located(
        (By.XPATH, '//tbody')))

        #Grab the page source
        new_html=driver.page_source

        html_list.append(new_html)

        #5 second cooldown 
        time.sleep(5)

    driver.quit()
except TimeoutError as tie:
    #Error Handling 
    print("Exception " +tie+ 'has occured')
    driver.quit()


In [39]:
soup = BeautifulSoup(html,'html.parser')

In [40]:
div=soup.select_one('table')

In [41]:
# t2.loc[:,['Name','Color','# Seen','IWD']]

In [106]:
div = soup.select_one('table')
t2 = pd.read_html(div.prettify())[0]
t2.head()

for idx, tbl in enumerate(html_list):
    soup = BeautifulSoup(tbl,'html.parser')
    div = soup.select_one('table')
    new_table = pd.read_html(div.prettify())[0].loc[:,['Name','IWD','# GP','# Seen']]
    #new_table['IWD'] = np.where((new_table['# Picked']>1700)&(new_table['Rarity']!='M'),new_table['IWD'],0)
    new_table['Seen_rate'] = new_table['# GP'] /new_table['# Seen'] 
    new_table = new_table.loc[:,['Name','Seen_rate']]
    t2 = t2.merge(right=new_table,how='inner',on='Name',suffixes=[None," " + archetypes[idx]])

In [93]:
# t2.loc[:,['Color','IWD','IWD WU', 'IWD WB', 'IWD WR', 'IWD WG', 'IWD UB', 'IWD UR',
#        'IWD UG', 'IWD BR', 'IWD BG', 'IWD RG']].columns.values

In [94]:
# tz = t2.loc[:,['Color','IWD WU', 'IWD WB', 'IWD WR', 'IWD WG', 'IWD UB', 'IWD UR',
#        'IWD UG', 'IWD BR', 'IWD BG', 'IWD RG']]

# tz.columns = tz.columns.str.replace(r'IWD', '')

In [95]:
# cols = tz.iloc[:,1:].columns.values
# for c in cols:
#     tz[c] = tz[c].str.replace('pp','').astype('float')

In [96]:
# #Let's use iterrows to rip through 
# cols = tz.columns.values
# lencols = tz.iloc[:,1:].columns.values

# #We will create arrays w 0 and 1 here to see what's good 
# master_list = []

# for index, row in tz.iterrows():
#     rowlist = []


#     row_color = row['Color']

#     #iterate through all indices 
#     for column in lencols:
#         print(row_color)
#         #now iterate through the 10 columns w color combos to find matches 
#         try:
#             if row_color in column:
#                 rowlist.append(1)
#             else:
#                 rowlist.append(0)

#         #Cards without colors 
#         except TypeError:
#             rowlist.append(0)
#     #Now that we have that, let's regex match to our cols array
#     master_list.append(rowlist)

In [110]:
# #This cell actually does things.... here we will impute null values using the card's avg IWD
# impute_colnames = t2.iloc[:,-len(archetypes):].columns.values
# t2copy = t2.copy()

# #Make all nulls np.nan so we can remove them with the min with a masked array
# t2copy['IWD'] = t2copy['IWD'].fillna('999.9pp')

# #Remove the pp suffix from our numbers and convert to float
# t2copy['IWD'] = t2copy['IWD'].str.replace('pp','').astype('float')

# #Create masked array that will look for all positions where we have np.nan and replace it with the min
# #The intuition here is if we don't have enough data for the IWD of a card at all, people are against picking it all, thus meaning it must have a low IWD
# masked_iwd = t2copy['IWD'].mask(
#     t2copy['IWD'].eq(999.9),
#     t2copy['IWD'].min())

# #Replace old IWD column with our masked_iwd, which is a float
# t2copy['IWD'] = masked_iwd


# for c in impute_colnames:

#     #Generally, we will fill the specialized cols with the overall IWD when the sample is small
#     t2copy[c] = t2copy[c].fillna(t2copy['IWD']).astype('string')

#     #For those we aren't replacing, we also need to clean off the 'pp' string and convert datatypes
#     t2copy[c] = t2copy[c].str.replace('pp','').astype('float')


tz = t2.iloc[:,-10:].columns.values
tz = np.append(tz,['Name','Color'])
tt = t2.loc[:,tz]


#Now we deal with the weights of the missing lands. 
# Since IWD refers to an increase in win percentage, it makes the most sense of the IWD of a land to be 0 
#Concept here being unless it hurts your deck, you should take another card

lands = ['Swamp', 'Forest', 'Island', 'Plains', 'Mountain']
colors =['B','G','U','W','R']
zeros = [0 for x in archetypes]

output = []
for idx, l in enumerate(lands):
    new_zeros = zeros.copy()
    new_zeros.append(l)
    new_zeros.append(colors[idx])
    output.append(new_zeros)

dummies_df = pd.DataFrame(output, columns=tt.columns.values)

final_weights_df = pd.concat([tt, dummies_df]).sort_values(by='Name').reset_index(drop=True)

In [111]:
final_weights_df.shape[0]

272

In [112]:
final_weights_df

,Seen_rate,Seen_rate WB,Seen_rate WR,Seen_rate WG,Seen_rate UB,Seen_rate UR,Seen_rate UG,Seen_rate BR,Seen_rate BG,Seen_rate RG,Name,Color
0,0.004976,0.006804,0.236857,0.006454,0.007504,0.423970,0.009380,0.575744,0.012495,0.460709,Abrade,R
1,0.073039,0.079949,0.066778,0.063529,0.000007,0.000000,0.000000,0.000072,0.000026,0.000046,Adamant Will,W
2,0.000000,0.010772,0.000000,0.000086,0.008072,0.000012,0.000000,0.014811,0.005340,0.000037,Aim for the Head,B
3,0.000768,0.000000,0.009029,0.000448,0.000320,0.104694,0.000896,0.011846,0.000000,0.010053,Alchemist's Gambit,UR
4,0.111030,0.000000,0.000000,0.000109,0.075116,0.125582,0.032008,0.000026,0.000077,0.000096,Alchemist's Retrieval,U
...,...,...,...,...,...,...,...,...,...,...,...,...
267,0.000000,0.000038,0.000000,0.046230,0.000082,0.000044,0.021633,0.000019,0.037892,0.113891,Witch's Web,G
268,0.040092,0.000704,0.000000,0.000289,0.046735,0.055996,0.032150,0.000361,0.000830,0.000758,Witness the Future,U
269,0.000461,0.000830,0.000164,0.169495,0.001567,0.000860,0.110074,0.001659,0.145489,0.345411,Wolf Strike,G
270,0.000000,0.000163,0.000391,0.139228,0.000000,0.000000,0.089855,0.000554,0.121088,0.515844,Wolfkin Outcast,G


In [99]:
final_weights_df.to_csv('VOM_Weights_default__seen_rates_df.csv', index=False)

In [100]:

final_weights_df.loc[:, ~final_weights_df.columns.isin(['Name', 'Color'])].to_numpy().reshape((272,10))

array([[  3. ,   3.3,   0.5, ...,   1.9,   7.5,   3. ],
       [ -1.3,   1. ,  -2.1, ..., -11.5, -11.5, -11.5],
       [-11.5,   0.8, -11.5, ...,  -2.1,  -4.1, -11.5],
       ...,
       [  1.2,   1.2,   1.2, ...,   1.2,   0.6,   1.2],
       [ -0.3,  -0.3,  -0.3, ...,  -0.3,  -0.4,  -0.3],
       [ -1.5, -11.5, -11.5, ..., -11.5, -11.5, -11.5]])

In [101]:
df = pd.read_csv("weights_data/VOM_Weights_max_and_zeroed.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'weights_data/VOM_Weights_max_and_zeroed.csv'

In [ ]:
df = df.iloc[:,]

In [ ]:
df

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9
0,0,0.000000,0.000000,1.849743,0.000000,0.0,13.688130,0.000000,3.797531,0.000000,10.567747
1,1,-1.885970,1.412178,-5.284979,-1.443827,-0.0,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000
2,2,-0.000000,-0.651775,-0.000000,-0.000000,-1.9,-0.000000,-0.000000,-6.883025,-7.148148,-0.000000
3,3,-0.000000,-0.000000,-0.000000,-0.000000,-0.0,0.448791,-0.000000,-0.000000,-0.000000,-0.000000
4,4,-1.595820,-0.000000,-0.000000,-0.000000,1.5,0.673187,6.344033,-0.000000,-0.000000,-0.000000
...,...,...,...,...,...,...,...,...,...,...,...
267,267,0.000000,0.000000,0.000000,-0.962551,0.0,0.000000,8.128292,0.000000,0.595679,0.340895
268,268,3.046566,-0.000000,-0.000000,-0.000000,0.9,-3.814725,3.766770,-0.000000,-0.000000,-0.000000
269,269,0.000000,0.000000,0.000000,1.684465,0.0,0.000000,9.714300,0.000000,1.588477,3.408951
270,270,-0.000000,-0.000000,-0.000000,-5.053395,-0.0,-0.000000,-3.568519,-0.000000,-0.794239,-0.681790
